In [1]:
import os
from dotenv import load_dotenv
import json

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import PromptTemplate

#### load env variables

In [3]:
_ = load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

#### Defining the tools

In [5]:
tool = TavilySearchResults(max_results=2,api_key=tavily_api_key)

In [6]:
memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
### TODO: create tools for the agent to use

#### Defining the agent

In [8]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph=StateGraph(AgentState)
        graph.add_node("llm", self.call_model)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm",
                                    self.exists_action,
                                    {True: "action", False: END})   
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model= model.bind_tools(tools)

    def call_model(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message=self.model.invoke(messages)
        return {'messages':[message]}
    def exists_action(self, state: AgentState):
        result=state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls=state["messages"][-1].tool_calls
        results=[]
        for t in tool_calls:
            print(f"calling tool : {t}")
            result = self.tools[t['name']].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t['id'],name=t['name'], content=str(result)))
        print("Back to the model")
        return {'messages':results}
    


### TODOs

- [ ] **_Task 1:_** change the prompt to analyze logs
- [ ] **_Task 2:_** Implement function Y
- [ ] **_Task 3:_** Test the implementation of Z

Remember to **_update_** this list as tasks are completed or new tasks arise.

In [9]:
system_message = """
You are a log analysis assistant. Your task is to analyze the provided JSON logs and extract meaningful insights. For each log entry, identify and summarize key details, including the event type, user actions, system warnings, and errors. Highlight any patterns or notable incidents.
"""

user_message_template_str = """
Here are the logs:

{logs}

Tasks:
1. **User Activities**: Summarize the actions performed by each user.
2. **System Warnings and Errors**: Identify and explain any warnings and errors logged.
3. **Patterns and Anomalies**: Highlight any patterns or unusual activities observed in the logs.

Output Format:
Provide a structured summary with sections for User Activities, System Warnings and Errors, and Patterns and Anomalies.
"""
user_message_template = PromptTemplate(input_variables=["logs"], template=user_message_template_str)


##### reading logs json file

In [12]:
with open("../data/logs.json","r") as f:
    logs = json.load(f)

logs_json = json.dumps(logs, indent=4)

In [14]:
user_message = user_message_template.format(logs=logs_json)

##### Models

In [15]:
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key, verbose=True)
# model = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)


In [16]:
abot = Agent(model, [], system=system_message, checkpointer=memory)

In [17]:
messages = [HumanMessage(content=user_message)]

##### Stream thread

In [18]:
# thread = {"configurable": {"thread_id": "3"}}
# for event in abot.graph.stream({"messages": messages}, thread):
#     for v in event.values():
#         print(v['messages'])

result= abot.graph.invoke({"messages":messages },
                     config = {"configurable":{"thread_id": "4"}})

In [19]:
analysis=result['messages'][-1].content

In [20]:
from IPython.display import display, Markdown

In [21]:
display(Markdown(analysis))

## Log Analysis Summary

### User Activities

**John Doe:**
* Logged in at 12:34:56Z with session ID "abc123" from IP 192.168.1.10.
* Attempted to read "/restricted/financial_report.xlsx" but was denied due to permission issues.
* Successfully read "/public/annual_report.pdf".
* Logged out at 12:37:22Z after a session duration of 146 seconds.

**Jane Smith:**
* Logged in at 12:45:00Z with session ID "def456" from IP 192.168.1.15.
* Successfully wrote to "/public/sales_data.csv".
* Logged out at 12:55:10Z after a session duration of 310 seconds.

### System Warnings and Errors

**Errors:**
* At 12:35:01Z, user "john_doe" was denied access to "/restricted/financial_report.xlsx" due to "Permission Denied".
* At 12:50:00Z, a "System Error" occurred with error code 500 and message "Internal Server Error" in the "Authentication Service".

**Warnings:**
* At 12:36:15Z, the system logged a warning about high resource utilization with CPU usage at 85% and memory usage at 70%.

### Patterns and Anomalies

* **Unauthorized Access Attempt:** John Doe attempted to access a restricted file ("/restricted/financial_report.xlsx"), which was denied. This indicates a potential security concern and requires further investigation.
* **High Resource Utilization:** The system warning at 12:36:15Z indicates a potential performance bottleneck. It's important to investigate the cause of high CPU and memory usage to ensure system stability.
* **System Error:** The "Internal Server Error" in the Authentication Service at 12:50:00Z suggests a possible bug or configuration issue. This error needs immediate attention to restore proper authentication functionality.
* **User Access Patterns:** Both users accessed public files. John Doe attempted to access a restricted file, indicating a potential security breach, while Jane Smith only accessed public files. This requires further analysis to understand the context of their actions.

**Further Investigation:**

* The "Permission Denied" error for John Doe accessing the restricted file should be investigated to determine if it was a legitimate attempt or a potential security breach.
* The "Internal Server Error" in the Authentication Service requires immediate attention and debugging to determine the root cause and prevent future occurrences.
* The high resource utilization should be investigated to identify the processes or services responsible and implement necessary optimizations.


##### Async thread

In [ ]:
# messages = [HumanMessage(content="What is the weather in Casablanca?")]
# thread = {"configurable": {"thread_id": "4"}}
# async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
#     kind = event["event"]
#     if kind == "on_chat_model_stream":
#         content = event["data"]["chunk"].content
#         if content:
#             # Empty content in the context of OpenAI means
#             # that the model is asking for a tool to be invoked.
#             # So we only print non-empty content
#             print(content, end="|")